In [2]:
from typing import *
import numpy as np
import torch
from exp_utils import get_data
import torch.nn as nn
from monotonic import MonotonicLinear
import torch
import numpy as np
from tqdm import trange

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
mask_mono = np.array([-1, 1, -1, -1, 1] + [0] * 23)

In [5]:
def split_input(inputs):
    return inputs[:, np.where(1-mask_mono)].squeeze(), \
        inputs[:, np.where(mask_mono)].squeeze() * torch.tensor(mask_mono[np.where(mask_mono)][None,:], dtype=torch.float32).to(device)

In [6]:
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor

train_df, val_df = get_data("loan")
X_train = torch.tensor(train_df.loc[:, train_df.columns != 'ground_truth'].values).to(device)
X_val = torch.tensor(val_df.loc[:, val_df.columns != 'ground_truth'].values).to(device)
y_train = torch.tensor(train_df.loc[:, train_df.columns == 'ground_truth'].values).to(device)
y_val = torch.tensor(val_df.loc[:, val_df.columns == 'ground_truth'].values).to(device)

train_loader = torch.utils.data.DataLoader(TensorDataset(X_train, y_train), batch_size=256, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(TensorDataset(X_val, y_val), batch_size=256, shuffle=True, drop_last=True)

Upload skipped, file /home/alberto_sinigaglia/jupyter_notebooks/inverseCDF/data/train_loan.csv exists.
Upload skipped, file /home/alberto_sinigaglia/jupyter_notebooks/inverseCDF/data/test_loan.csv exists.


In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    losses = []
    val_losses = []
    best_val_acc = 0.
    for epoch in trange(num_epochs):
        model.train()
        total = 0
        losses_buffer = []
        for inputs, labels in train_loader:
            inputs_free, inputs_mono = split_input(inputs)
            optimizer.zero_grad()
            outputs = model(inputs_free.float(), inputs_mono.float())
            loss = criterion(outputs, labels.float())
            losses_buffer.append(loss)
            loss.backward()
            optimizer.step()
            
            total += labels.size(0)
        losses.append(np.mean([el.detach().cpu() for el in losses_buffer]))
        
        val_acc, val_loss = validate_model(model, val_loader, criterion)
        best_val_acc = max(best_val_acc, val_acc)
        val_losses.append(val_loss)
    
    return losses, val_losses, best_val_acc

def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = []
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs_free, inputs_mono = split_input(inputs)
            outputs = model(inputs_free.float(), inputs_mono.float())
            loss = criterion(outputs, labels.float())
            val_loss += [loss.item()]

            total += labels.size(0)
            correct += (outputs.round() == labels).sum().item()

    val_accuracy = correct / total
    return val_accuracy, np.mean(val_loss)

In [8]:
class ReLUMonoModel(torch.nn.Module):
    def __init__(self, input_size_mono, num_layers_mono, num_layers_pre_mono, num_neurons_mono, num_neurons_pre_mono) -> None:
        super().__init__()
        self.pre_mono = torch.nn.ModuleList([torch.nn.LazyLinear(num_neurons_pre_mono) for _ in range(num_layers_pre_mono)])
        self.mono = torch.nn.ModuleList(
            [
                MonotonicLinear(input_size_mono + num_neurons_pre_mono, num_neurons_mono, pre_activation=nn.Identity()),
                *[MonotonicLinear(num_neurons_mono, num_neurons_mono, pre_activation=nn.ReLU()) for _ in range(num_layers_mono)],
                MonotonicLinear(num_neurons_mono, 1, pre_activation=nn.ReLU()),
            ]
        )
    def forward(self, x, x_mono):
        for layer in self.pre_mono:
            x = torch.nn.functional.relu(layer(x))
        
        x = torch.cat((x, x_mono), dim=-1)
        for layer in self.mono:
            x = layer(x)
        
        return torch.nn.functional.sigmoid(x)
        

In [9]:
model = ReLUMonoModel((mask_mono!=0).sum(),3,3, 32,32).to(device)

In [10]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
losses, val_losses, best_val_accuracy= train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

100%|██████████| 50/50 [07:12<00:00,  8.65s/it]


In [11]:
best_val_accuracy

0.6529567746350365